In [1]:
import re
import pandas as pd
from nltk import tokenize

In [2]:
# -*- coding: utf-8 -*-

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

这里dataframe自带的index和id对不上是因为，前面merge里drop了na的row

In [3]:
data_df = pd.read_csv('../../data/cr_sec_ori/corporate_rating_with_cik_and_sec_merged_text.csv')
# data_df = data_df.set_index('Id')
data_df

,Id,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,...,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,CIK,secText
0,0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,...,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655,106640,\nABOUT WHIRLPOOL\n\nWhirlpool is the number o...
1,1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,...,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846,106640,\nABOUT WHIRLPOOL\n\nWhirlpool Corporation (Wh...
2,2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,...,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510,106640,\nABOUT WHIRLPOOL\n\nWhirlpool Corporation (Wh...
3,3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,...,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150,106640,\nABOUT WHIRLPOOL\n\nWhirlpool Corporation (Wh...
4,4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,...,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,106640,"\nABOUT WHIRLPOOL\n\nWhirlpool Corporation (""W..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,2020,AA,"Gartner, Inc.",IT,Egan-Jones Ratings Company,2/18/2016,Consumer Services,0.862111,0.720623,0.281812,...,4.121082,4.448027,-16.425121,0.125846,23.234839,4.121082,0.159756,2.164291,749251,\nThe purpose of the following Managements Dis...
1797,2021,B,SBA Communications Corporation,SBAC,Standard & Poor's Ratings Services,8/28/2013,Consumer Services,0.459629,0.266021,0.163323,...,2.572565,0.998349,19.002336,-0.043850,22.537466,3.894427,0.381332,16.799152,1034054,\nWe are a leading independent owner and opera...
1798,2023,AA,Imperial Oil Limited,IMO,DBRS,10/7/2016,Energy,0.986608,0.886943,0.100695,...,1.111373,1.676498,1.664762,0.089347,7.977219,2.377301,0.073664,0.000000,49938,Item 2. Managements Discussion and Analysis of...
1799,2024,BBB,"NVR, Inc.",NVR,Moody's Investors Service,9/5/2012,Capital Goods,11.757767,7.115059,7.057420,...,50.293155,230.194971,1.759461,0.086376,15.867701,52.760726,0.083018,15.758348,906163,Item 2. Managements Discussion and Analysis of...


In [4]:
split_sec_data_df = pd.DataFrame(columns=data_df.columns)
split_sec_data_df

,Id,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,...,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,CIK,secText


In [5]:
import warnings

# import nltk
# nltk.download('punkt')
warnings.filterwarnings('ignore')

In [6]:
def split_sec_to_sent_create_new_df(df):
    rows = []
    for index, row in df.iterrows():
        except_txt = row[:-1]
        sec_txt = row['secText']
        # sentences = split_into_sentences(sec_txt)
        sentences = tokenize.sent_tokenize(sec_txt)
        for sent in sentences:
            except_txt['secText'] = sent
            # split_sec_data_df = pd.concat([split_sec_data_df, except_txt], ignore_index=True)
            # split_sec_data_df = split_sec_data_df.append(except_txt, ignore_index=True)
            rows.append(except_txt)
    return rows

In [7]:
import itertools
from multiprocessing import Pool

processes = 12

num_row = len(data_df)
multi_thread_tasks = []
process_pool = Pool(processes=processes,)
task_batch_size = int(num_row / processes + 1)
multi_thread_tasks = []
pno = 0
for t in range(0, num_row, task_batch_size):
    batch_tasks = data_df.loc[t:t + task_batch_size]
    multi_thread_tasks.append(process_pool.apply_async(split_sec_to_sent_create_new_df,
                                                       (batch_tasks,),
                                                       # callback=lambda x: pbar.update(task_batch_size),
                                                       ))
    pno += 1

exec_res = [t.get() for t in multi_thread_tasks]
res = list(itertools.chain(*exec_res))
process_pool.close()  # 关闭线程池清理

In [8]:
split_sec_data_df = pd.concat(res, ignore_index=True)

In [9]:
split_sec_data_df.to_csv('../../data/cr_sec_ori/corporate_rating_with_cik_and_sec_sentences.csv')


KeyboardInterrupt

